<a href="https://colab.research.google.com/github/ushaprof/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [ ]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [ ]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [ ]:
y_df.value_counts()

Attrition  Department            
No         Research & Development    828
           Sales                     354
Yes        Research & Development    133
           Sales                      92
No         Human Resources            51
Yes        Human Resources            12
Name: count, dtype: int64

In [ ]:
# Create a list of at least 10 column names to use as X data
select_columns = ['Age','DistanceFromHome','Education', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance','YearsAtCompany','YearsSinceLastPromotion','NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[select_columns]
X_df.head()


# Show the data types for X_df
X_df.dtypes



,0
Age,int64
DistanceFromHome,int64
Education,int64
JobSatisfaction,int64
OverTime,object
StockOptionLevel,int64
WorkLifeBalance,int64
YearsAtCompany,int64
YearsSinceLastPromotion,int64
NumCompaniesWorked,int64


In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)


In [ ]:
# Convert your X data to numeric data types however you see fit
from sklearn.preprocessing import OneHotEncoder
OverTime_encoder = OneHotEncoder(sparse_output=False,drop='first', handle_unknown='ignore')
OverTime_encoder.fit(np.array(X_train['OverTime']).reshape(-1,1))
X_train['OverTimeEncoded'] = OverTime_encoder.transform(np.array(X_train['OverTime']).reshape(-1,1))
X_test['OverTimeEncoded'] = OverTime_encoder.transform(np.array(X_test['OverTime']).reshape(-1,1))
X_train.drop(columns=['OverTime'], inplace=True)
X_test.drop(columns=['OverTime'], inplace=True)
X_train.head()




,Age,DistanceFromHome,Education,JobSatisfaction,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked,OverTimeEncoded
591,33,16,3,1,0,3,3,0,5,0.0
267,25,5,2,1,2,3,6,1,1,0.0
1236,36,13,5,1,3,3,2,2,5,1.0
788,28,10,3,3,0,4,8,1,3,0.0
1224,26,17,4,3,3,4,3,0,1,0.0


In [ ]:
# Create a StandardScaler
scaler = StandardScaler()



# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)



# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
Department_encoder = OneHotEncoder(sparse_output=False,drop=None, handle_unknown='ignore')



# Fit the encoder to the training data
Department_encoder.fit(np.array(y_train['Department']).reshape(-1,1))


# Create two new variables by applying the encoder
# to the training and testing data
y_department_train_encoded = Department_encoder.transform(np.array(y_train['Department']).reshape(-1,1))
y_department_test_encoded = Department_encoder.transform(np.array(y_test['Department']).reshape(-1,1))
y_department_train_encoded


array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [ ]:
# Create a OneHotEncoder for the Attrition column
Attrition_encoder = OneHotEncoder(sparse_output=False,drop=None, handle_unknown='ignore')


# Fit the encoder to the training data
Attrition_encoder.fit(np.array(y_train['Attrition']).reshape(-1,1))


# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train_encoded = Attrition_encoder.transform(np.array(y_train['Attrition']).reshape(-1,1))
y_attrition_test_encoded = Attrition_encoder.transform(np.array(y_test['Attrition']).reshape(-1,1))
y_attrition_train_encoded


array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

## Part 2: Create, Compile, and Train the Model

In [ ]:
# Find the number of columns in the X training data.
number_input_features = len(X_train.columns)
number_input_features


# Create the input layer
input_layer = layers.Input(shape=(number_input_features,), name='input_layer')


# Create at least two shared layers
shared_layer_1 = layers.Dense(units=64, activation='relu', name='shared_layer_1')(input_layer)
shared_layer_2 = layers.Dense(units=128, activation='relu', name='shared_layer_2')(shared_layer_1)


In [ ]:
# Create a branch for Department
# with a hidden layer and an output layer


# Create the hidden layer
department_hidden_layer = layers.Dense(units=32, activation='relu', name='department_hidden_layer')(shared_layer_2)


# Create the output layer
department_output_layer = layers.Dense(y_department_train_encoded.shape[1], activation='sigmoid', name='department_output_layer')(department_hidden_layer)


In [ ]:
# Create a branch for Attrition
# with a hidden layer and an output layer


# Create the hidden layer
attrition_hidden_layer = layers.Dense(units=32, activation='relu', name='attrition_hidden_layer')(shared_layer_2)


# Create the output layer
attrition_output_layer = layers.Dense(y_attrition_train_encoded.shape[1], activation='sigmoid', name='attrition_output_layer')(attrition_hidden_layer)


In [ ]:
# Create the model
model = Model(inputs=input_layer, outputs={'department_output_layer':department_output_layer, 'attrition_output_layer':attrition_output_layer})



# Compile the model
model.compile(optimizer='adam',
              loss={
                  'department_output_layer': 'categorical_crossentropy',
                  'attrition_output_layer': 'binary_crossentropy'
              },
              metrics={
                  'department_output_layer': 'accuracy',
                  'attrition_output_layer': 'accuracy'
              })




# Summarize the model
model.summary()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer_1 (Dense)    │ (None, 64)             │            704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer_2 (Dense)    │ (None, 128)            │          8,320 │ shared_layer_1[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_hidden_layer    │ (None, 32)             │          4,128 │ shared_layer_2[0][0]   │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_hidden_layer   │ (None, 32)             │          4,128 │ shared_layer_2[0][0]   │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output_layer    │ (None, 2)              │             66 │ attrition_hidden_laye… │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output_layer   │ (None, 3)              │             99 │ department_hidden_lay… │
│ (Dense)                   │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
fit_model = model.fit(
    X_train_scaled,
    {'department_output_layer': y_department_train_encoded, 'attrition_output_layer': y_attrition_train_encoded},epochs=100)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - attrition_output_layer_accuracy: 0.5994 - attrition_output_layer_loss: 0.6506 - department_output_layer_accuracy: 0.5035 - department_output_layer_loss: 0.9961 - loss: 1.6468
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_layer_accuracy: 0.8111 - attrition_output_layer_loss: 0.4942 - department_output_layer_accuracy: 0.6674 - department_output_layer_loss: 0.7627 - loss: 1.2569
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_layer_accuracy: 0.8017 - attrition_output_layer_loss: 0.4633 - department_output_layer_accuracy: 0.6783 - department_output_layer_loss: 0.7414 - loss: 1.2048
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - attrition_output_layer_accuracy: 0.8278 - attrition_output_layer_loss: 0.4255 - department_output_layer_accuracy: 0.6715 - department_output_layer_loss: 0.7282 - loss: 1.1537
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_layer_accuracy: 0.

In [ ]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output_layer': y_department_test_encoded, 'attrition_output_layer': y_attrition_test_encoded})
print (test_results)

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - attrition_output_layer_accuracy: 0.8162 - attrition_output_layer_loss: 1.0518 - department_output_layer_accuracy: 0.4770 - department_output_layer_loss: 3.9013 - loss: 4.9561
[4.862035751342773, 1.1177517175674438, 3.725008249282837, 0.8070651888847351, 0.46739131212234497]


In [ ]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[4]}")
print(f"Attrition Accuracy: {test_results[3]}")




Department Accuracy: 0.46739131212234497
Attrition Accuracy: 0.8070651888847351


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

## Part 3: Summary

### 1. Is accuracy the best metric to use on this data? Why or why not?
Accuracy is often used but may not be the best metric, especially if the target variables are imbalanced. Other metrics such as precision, recall, or F1-score may provide more meaningful insights.

### 2. What activation functions did you choose for your output layers, and why?
I used the **sigmoid** activation function for both output layers. Sigmoid is suitable for binary classification, as it outputs a probability between 0 and 1, making it a good choice for categorical outcomes with two classes (e.g., 'Yes' or 'No').

### 3. Can you name a few ways that this model could be improved?
- Use techniques like **oversampling** or **undersampling** if there's class imbalance.
- Perform **hyperparameter tuning** to optimize model performance.
- Use more advanced algorithms like **decision trees** or **gradient boosting** for better classification.


